In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import glob
import os
import plotly.express as px

#### Loading the irradiance data from the DUETT project

In [92]:
irr_df=pd.read_csv(r'D:\thesis data files\Cologne_irradiance_stundenwerte_duett_FG_18209_akt\produkt_fg_duett_20240219_20250821_18209.txt', sep=';', header=0, parse_dates=['MESS_DATUM'], dayfirst=True, na_values=-999)
irr_df["MESS_DATUM"] = pd.to_datetime(irr_df["MESS_DATUM"], format="%Y%m%d%H")


C:\Users\ashka\AppData\Local\Temp\ipykernel_14528\3785022206.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



##### Removing the non necessary columns 


In [93]:
irr_df1=irr_df.drop(columns=['STATIONS_ID','eor'])
irr_df1


,MESS_DATUM,QN_952,FG_DUETT,FG_UN_DUETT
0,2024-04-01 01:00:00,500,0,NaN
1,2024-04-01 02:00:00,500,0,NaN
2,2024-04-01 03:00:00,500,0,NaN
3,2024-04-01 04:00:00,500,0,NaN
4,2024-04-01 05:00:00,502,0,NaN
...,...,...,...,...
12186,2025-08-21 19:00:00,503,9,1.0
12187,2025-08-21 20:00:00,500,0,0.0
12188,2025-08-21 21:00:00,500,0,0.0
12189,2025-08-21 22:00:00,500,0,0.0


##### filtering the data only for the period for which we have observed gen data form the TH i.e. from 2025-04-15 07:00:00 to 2025-08-19 00:00:00

In [99]:
irr_df3=irr_df1[(irr_df1['MESS_DATUM']>='2025-04-15 07:00:00') & (irr_df1['MESS_DATUM']<='2025-08-19 00:00:00')].copy()

irr_df3

,MESS_DATUM,QN_952,FG_DUETT,FG_UN_DUETT
9102,2025-04-15 07:00:00,503,61,12.0
9103,2025-04-15 08:00:00,503,93,15.0
9104,2025-04-15 09:00:00,503,90,50.0
9105,2025-04-15 10:00:00,503,132,21.0
9106,2025-04-15 11:00:00,503,201,10.0
...,...,...,...,...
12115,2025-08-18 20:00:00,502,0,0.0
12116,2025-08-18 21:00:00,500,0,0.0
12117,2025-08-18 22:00:00,500,0,0.0
12118,2025-08-18 23:00:00,500,0,0.0


In [100]:
irr_df3.rename(columns={'FG_DUETT':'FG_DUETT (J/cm²)','FG_UN_DUETT':'Hourly_FG_DUETT (J/cm²)'}, inplace=True)
irr_df3

,MESS_DATUM,QN_952,FG_DUETT (J/cm²),Hourly_FG_DUETT (J/cm²)
9102,2025-04-15 07:00:00,503,61,12.0
9103,2025-04-15 08:00:00,503,93,15.0
9104,2025-04-15 09:00:00,503,90,50.0
9105,2025-04-15 10:00:00,503,132,21.0
9106,2025-04-15 11:00:00,503,201,10.0
...,...,...,...,...
12115,2025-08-18 20:00:00,502,0,0.0
12116,2025-08-18 21:00:00,500,0,0.0
12117,2025-08-18 22:00:00,500,0,0.0
12118,2025-08-18 23:00:00,500,0,0.0


#### loading the windspeed data 

In [123]:
wind_df=pd.read_csv(r'D:\thesis data files\\Cologne_wind_stundenwerte_FF_02667_akt\produkt_ff_stunde_20240318_20250918_02667.txt', sep=';', header=0, parse_dates=['MESS_DATUM'], dayfirst=True, na_values=[-999,'eor'])
wind_df["MESS_DATUM"] = pd.to_datetime(wind_df["MESS_DATUM"], format="%Y%m%d%H")

C:\Users\ashka\AppData\Local\Temp\ipykernel_14528\1550323829.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [124]:
wind_df.rename(columns={'   F':'Wind_Speed (m/s)','   D':'Wind_direction(degrees)'}, inplace=True)
wind_df.drop(columns=['STATIONS_ID','eor'], inplace=True)
wind_df1=wind_df[(wind_df['MESS_DATUM']>='2025-04-15 07:00:00') & (wind_df['MESS_DATUM']<='2025-08-19 00:00:00')].copy()
wind_df1

,MESS_DATUM,QN_3,Wind_Speed (m/s),Wind_direction(degrees)
9439,2025-04-15 07:00:00,3,1.1,150.0
9440,2025-04-15 08:00:00,3,2.4,140.0
9441,2025-04-15 09:00:00,3,3.8,140.0
9442,2025-04-15 10:00:00,3,2.8,180.0
9443,2025-04-15 11:00:00,3,3.6,190.0
...,...,...,...,...
12449,2025-08-18 20:00:00,1,2.0,100.0
12450,2025-08-18 21:00:00,1,2.4,90.0
12451,2025-08-18 22:00:00,1,2.1,100.0
12452,2025-08-18 23:00:00,1,1.5,120.0


In [125]:


wind_fig=px.line(wind_df1, x='MESS_DATUM', y='Wind_Speed (m/s)', title='Wind Speed over Time')
wind_fig.show()

#### there seems to be some data missing along with missing indices/datetime

In [127]:
expected_index = pd.date_range(start=wind_df1['MESS_DATUM'].min(), end=wind_df1['MESS_DATUM'].max(), freq='H')
missing_timestamps = expected_index.difference(wind_df1['MESS_DATUM'])
print(missing_timestamps)


C:\Users\ashka\AppData\Local\Temp\ipykernel_14528\2197311402.py:1: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



DatetimeIndex(['2025-05-31 11:00:00', '2025-05-31 12:00:00',
               '2025-05-31 13:00:00'],
              dtype='datetime64[ns]', freq='h')


In [129]:
wind_df1['Wind_Speed (m/s)'].isnull().sum()

34

##### Loading the observed genration data from the TH 

In [35]:
# th_df=pd.read_csv(r'D:\thesis data files\PV_gen_data_THK_2025\PV_THK_2025\Energie und Leistung - PV - Woche - Referenzanlage - 2025-04-15 - 2025-04-21.csv', sep=';', header=11 )

# th_df

In [88]:
folder_path = r'D:\thesis data files\PV_gen_data_THK_2025\PV_THK_2025'
file_pattern = os.path.join(folder_path, '*.csv')
# Empty list to hold individual DataFrames
df_list = []

# Loop through each file and read into DataFrame using the same parameters
for file in glob.glob(file_pattern):
    df = pd.read_csv(file, sep=';',decimal=',', header=11)
    df_list.append(df)

# Concatenate all DataFrames vertically
combined_df = pd.concat(df_list, ignore_index=True)

##### performing some data cleaning and manipulation

In [89]:
combined_df_1=combined_df.copy()
combined_df_1['Zeitraum'] = pd.to_datetime(combined_df_1['Zeitraum'])
combined_df_1 = combined_df_1.set_index('Zeitraum')
combined_df_1['Leistung [W]'] = pd.to_numeric(combined_df_1['Leistung [W]'].astype(str).str.replace('.','',regex=False), errors='coerce').fillna(0)
hourly_combined_df_1= combined_df_1.resample('h').mean()
fig=px.line(hourly_combined_df_1, x=hourly_combined_df_1.index, y='Leistung [W]', title='Hourly Average PV Power Generation')
fig.show()

C:\Users\ashka\AppData\Local\Temp\ipykernel_14528\1989070202.py:2: UserWarning:

Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

